In [1]:
# ==== Reels 유해온도 산출(1% → +0.035℃, 36.5~40.0) : 단일 셀 실행용 ====
import os, json, re, unicodedata
import pandas as pd

# 0) 경로 설정 (여기만 네 환경에 맞게 바꿔!)
INPUT_JSONL    = "C:/Users/asia/Desktop/파이널프로젝트/03_데이터전처리/138개 릴스 댓글정보.jsonl"   # 수집해둔 JSONL
BAD_WORDS_TXT  = "C:/workspaces/FinalProject/bad_words.txt"            # 네가 만든 욕설 리스트
OUTPUT_CSV     = "C:/Users/asia/Desktop/파이널프로젝트/03_데이터전처리/reels_temperature.csv"     # 결과 저장 경로

# 1) 유틸
def load_badwords(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"bad_words.txt 파일을 못 찾음: {path}")
    with open(path, "r", encoding="utf-8") as f:
        words = [w.strip() for w in f if w.strip() and not w.startswith("#")]
    return sorted(set(words))

def compile_patterns(badwords):
    pats = []
    for w in badwords:
        # 문자 사이에 특수문자/공백 끼워넣기(우회) 대응: '시발' -> 시[\W_]*발
        esc = "".join([re.escape(ch) + r"[\W_]*" for ch in w])
        pats.append(re.compile(esc, re.IGNORECASE))
    return pats

def normalize(txt: str) -> str:
    t = unicodedata.normalize("NFKC", str(txt)).lower()
    t = re.sub(r"\s+", " ", t).strip()
    return t

def is_toxic(text: str, patterns) -> bool:
    t = normalize(text)
    return any(p.search(t) for p in patterns)

def pct_round_1(n_toxic: int, n_total: int) -> int:
    if n_total <= 0: return 0
    return int(round((n_toxic / n_total) * 100))  # 1% 단위 반올림

def temp_from_pct(pct: int) -> float:
    temp = 36.5 + pct * 0.035
    return round(min(temp, 40.0), 1)  # 상한 40.0, 소수1자리

# 2) 데이터 로드 (JSONL: 각 줄이 {"reels_index":…, "text":…})
records = []
with open(INPUT_JSONL, "r", encoding="utf-8") as f:
    for lineno, line in enumerate(f, 1):
        line = line.strip()
        if not line: 
            continue
        try:
            obj = json.loads(line)
        except Exception:
            # 혹시 파이썬 dict 문자열로 저장된 경우 대응
            obj = eval(line)
        rid  = obj.get("reels_index") or obj.get("reel_index") or obj.get("id")
        text = obj.get("text") or obj.get("comment") or ""
        if rid is None:
            print(f"[경고] {lineno}번째 줄: reels_index 없음 → 스킵")
            continue
        records.append({"reels_index": rid, "text": str(text)})

df = pd.DataFrame(records)
if df.empty:
    raise ValueError("입력 데이터가 비어있음. JSONL 구조(reels_index, text) 확인 필요")

# 3) bad_words 로드/컴파일
badwords = load_badwords(BAD_WORDS_TXT)
patterns = compile_patterns(badwords)

# 4) reels_index별 집계 → 유해온도
grouped = df.groupby("reels_index")["text"].apply(list).reset_index(name="comments")
rows = []
for _, row in grouped.iterrows():
    comments = row["comments"]
    flags = [is_toxic(t, patterns) for t in comments]
    n_total, n_toxic = len(comments), sum(flags)
    pct  = pct_round_1(n_toxic, n_total)
    temp = temp_from_pct(pct)
    rows.append({"reels_index": row["reels_index"], "temperature": temp})

out_df = pd.DataFrame(rows).sort_values("reels_index").reset_index(drop=True)

# 5) CSV 저장 (A: reels_index, B: temperature)
out_df.to_csv(OUTPUT_CSV, index=False)

# 6) 확인용 출력
print("저장 완료 →", OUTPUT_CSV)
print(out_df.head(10).to_string(index=False))


저장 완료 → C:/Users/asia/Desktop/파이널프로젝트/03_데이터전처리/reels_temperature.csv
 reels_index  temperature
           1         36.5
           2         36.5
           3         36.5
           4         36.5
           5         36.5
           6         36.5
           7         36.6
           8         36.5
           9         36.5
          10         36.5
